In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from random import *

In [4]:
# load data
data_train = pd.read_csv('./data/fashion-mnist_train.csv', header = 0)

In [5]:
data_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
labels = data_train['label'].values.reshape(1, 60000) # 60000 labels
train = data_train.drop('label', axis=1).transpose() # 784 features

In [7]:
data_train['label']

0        2
1        9
2        6
3        0
4        3
        ..
59995    9
59996    1
59997    8
59998    8
59999    7
Name: label, Length: 60000, dtype: int64

In [8]:
data_train['label'].shape

(60000,)

In [9]:
train = np.array(train / 255.0) # normalize vì giá trị của pixel nằm trong khoảng 0-255 nên chia cho 255 để chuẩn hóa về 0-1

In [10]:
# Building a Model with tensorflow
labels_ = np.zeros((60000, 10)) # 60000 labels, 10 classes (0-9)
labels_[np.arange(60000), labels] = 1 # one hot encoding là cách biểu diễn dữ liệu dạng số thành dạng vector, ví dụ 1 số 5 sẽ được biểu diễn thành vector [0,0,0,0,0,1,0,0,0,0]
labels_ = labels_.transpose() # 10 labels, 60000 examples là cách biểu diễn của tensorflow, transpose để đổi chỗ 2 chiều
labels_ = np.array(labels_) # chuyển về dạng numpy array
# mỗi một giá trị biểu thị cho một cột trong matrix
print(labels_)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [10]:
# The tensor flow Model
from tensorflow.python.framework import ops
n_dim = 784 # 28 x 28 pixels là kích thước của ảnh
ops.reset_default_graph() # reset graph để chạy lại model
# Number of neurons in the layers là số lượng nút ẩn
n1 = 5 # Number of neurons in layer 1 là số lượng nút ẩn của lớp 1
n2 = 10 # Number of neurons in output layer là số lượng nút ẩn của lớp 2

In [14]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

cost_history = np.empty(shape=[1], dtype = float) # cost history là lịch sử chi phí
learning_rate = tf.placeholder(tf.float32, shape=()) # learning rate là tốc độ học
X = tf.placeholder(tf.float32, [n_dim, None]) # placeholder cho dữ liệu đầu vào
Y = tf.placeholder(tf.float32, [10, None]) # placeholder cho dữ liệu đầu ra
W1 = tf.Variable(tf. truncated_normal ([n1, n_dim], stddev=.1)) # khởi tạo trọng số cho lớp 1
b1 = tf.Variable(tf.zeros([n1,1])) # khởi tạo bias cho lớp 1
W2 = tf.Variable(tf. truncated_normal ([n2, n1], stddev=.1)) # khởi tạo trọng số cho lớp 2
b2 = tf.Variable(tf.zeros([n2,1])) # khởi tạo bias cho lớp 2
# Let's build our network...
Z1 = tf.nn.relu(tf.matmul(W1, X) + b1) # tính toán giá trị đầu ra của lớp 1
Z2 = tf.nn.relu(tf.matmul(W2, Z1) + b2) # tính toán giá trị đầu ra của lớp 2
y_ = tf.nn.softmax(Z2,0) # tính toán giá trị đầu ra của lớp 3
cost = - tf.reduce_mean(Y * tf.log(y_)+(1-Y) * tf.log(1-y_)) # tính toán hàm mất mát
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost) # tối ưu hàm mất mát
init = tf.global_variables_initializer() # khởi tạo biến toàn cục


Instructions for updating:
non-resource variables are not supported in the long term


In [16]:
sess = tf.Session() # khởi tạo phiên làm việc
sess.run(tf.global_variables_initializer()) # khởi tạo biến toàn cục
training_epochs = 5000 # số lần lặp lại quá trình học
cost_history = [] # lịch sử chi phí
for epoch in range(training_epochs+1): # lặp lại quá trình học
 sess.run(optimizer, feed_dict = {X: train, Y: labels_, learning_rate: 0.001}) # tối ưu hàm mất mát
 cost_ = sess.run(cost, feed_dict={ X:train, Y: labels_, learning_rate: 0.001}) # tính toán hàm mất mát
 cost_history = np.append(cost_history, cost_) # lưu lại lịch sử chi phí
 if (epoch % 20 == 0):
    print("Reached epoch",epoch,"cost J =", cost_) # in ra chi phí sau mỗi 20 lần lặp lại quá trình học

Reached epoch 0 cost J = 0.3253602
Reached epoch 20 cost J = 0.3122124
Reached epoch 40 cost J = 0.2956843
Reached epoch 60 cost J = 0.27464214
Reached epoch 80 cost J = 0.25272465
Reached epoch 100 cost J = 0.22746854
Reached epoch 120 cost J = 0.209841
Reached epoch 140 cost J = 0.1988092
Reached epoch 160 cost J = 0.1909531
Reached epoch 180 cost J = 0.1849414
Reached epoch 200 cost J = 0.18018371
Reached epoch 220 cost J = 0.17624068
Reached epoch 240 cost J = 0.17287105
Reached epoch 260 cost J = 0.16998734
Reached epoch 280 cost J = 0.16751076
Reached epoch 300 cost J = 0.1653664
Reached epoch 320 cost J = 0.16349007
Reached epoch 340 cost J = 0.16182916
Reached epoch 360 cost J = 0.1603522
Reached epoch 380 cost J = 0.15902013
Reached epoch 400 cost J = 0.1578073
Reached epoch 420 cost J = 0.15669315
Reached epoch 440 cost J = 0.1556555
Reached epoch 460 cost J = 0.15469404
Reached epoch 480 cost J = 0.15363456
Reached epoch 500 cost J = 0.15220426
Reached epoch 520 cost J = 0.1

In [17]:
correct_predictions = tf.equal(tf.argmax(y_,0), tf.argmax(Y,0)) # tính toán số lượng dự đoán đúng, tf.argmax(y_,0) là giá trị dự đoán khả năng cao nhất của mỗi ảnh, tf.argmax(Y,0) là giá trị thực tế của mỗi ảnh
# hàm tf.equal() trả về True nếu 2 giá trị bằng nhau và False nếu 2 giá trị khác nhau
# hàm tf.argmax() trả về chỉ số của giá trị lớn nhất trong một vector
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float")) # tính toán độ chính xác, hàm tf.cast() chuyển đổi giá trị True thành 1 và False thành 0
print ("Accuracy:", accuracy.eval({X: train, Y: labels_, learning_rate: 0.001}, session = sess)) # in ra độ chính xác

Accuracy: 0.1


In [19]:
data_dev = pd.read_csv('fashion-mnist_test.csv', header = 0) # đọc dữ liệu test
labels_dev = data_dev['label'].values.reshape(1, 10000) # lấy nhãn của dữ liệu test
labels_dev_ = np.zeros((10000, 10)) # khởi tạo mảng 0 với kích thước 10000x10
labels_dev_[np.arange(10000), labels_dev] = 1 # chuyển đổi nhãn thành one-hot encoding, ví dụ nhãn 2 sẽ được chuyển thành [0,0,1,0,0,0,0,0,0,0]
labels_dev_ = labels_dev_.transpose() # chuyển vị ma trận
dev = data_dev.drop('label', axis=1).transpose() # lấy dữ liệu test và chuyển vị ma trận

In [20]:
correct_predictions = tf.equal(tf.argmax(y_,0), tf.argmax(Y,0))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))
print ("Accuracy:", accuracy.eval({X: dev, Y: labels_dev_, learning_rate:
0.001}, session = sess))

Accuracy: 0.1


In [21]:
# Batch Gradient Descent là một thuật toán tối ưu hàm mất mát, nó sẽ lặp lại quá trình học cho đến khi hàm mất mát không thể giảm nữa
sess = tf.Session()
sess.run(tf.global_variables_initializer())
training_epochs = 100
cost_history = []
for epoch in range(training_epochs+1):
 sess.run(optimizer, feed_dict = {X: train, Y: labels_, learning_rate:0.01})
 cost_ = sess.run(cost, feed_dict={ X:train, Y: labels_, learning_rate:0.01})
 cost_history = np.append(cost_history, cost_)
 if (epoch % 50 == 0):
    print("Reached epoch",epoch,"cost J =", cost_)

Reached epoch 0 cost J = 0.322951
Reached epoch 50 cost J = 0.21640286
Reached epoch 100 cost J = 0.20742059


In [22]:
correct_predictions = tf.equal(tf.argmax(y_,0), tf.argmax(Y,0))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))
print ("Accuracy:", accuracy.eval({X: train, Y: labels_, learning_rate:0.001}, session = sess))


Accuracy: 0.45683333


In [ ]:
# Stochastic Gradient Descent

In [23]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
cost_history = []
for epoch in range(100+1):
    for i in range(0, 60000, 100):
        X_train_mini = train[:, i:i+100]
        y_train_mini = labels_[:, i:i+100]
        sess.run(optimizer, feed_dict = {X: X_train_mini, Y: y_train_mini, learning_rate: 0.0001})
        cost_ = sess.run(cost, feed_dict={ X:X_train_mini, Y: y_train_mini, learning_rate: 0.0001})
    cost_history = np.append(cost_history, cost_)
    if (epoch % 50 == 0):
        print("Reached epoch",epoch,"cost J =", cost_)

Reached epoch 0 cost J = 0.289533
Reached epoch 50 cost J = 0.19098035
Reached epoch 100 cost J = nan


In [25]:
# Accuracy
correct_predictions = tf.equal(tf.argmax(y_,0), tf.argmax(Y,0))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))
print ("Accuracy:", accuracy.eval({X: train, Y: labels_, learning_rate:0.001}, session = sess))

Accuracy: 0.35521665


In [24]:
# Mini-Batch Gradient Descent
sess = tf.Session()
sess.run(tf.global_variables_initializer())
cost_history = []
for epoch in range(100+1):
    for i in range(0, 60000, 50):
        X_train_mini = train[:, i:i+50]
        y_train_mini = labels_[:, i:i+50]
    sess.run(optimizer, feed_dict = {X: X_train_mini,Y: y_train_mini,learning_rate: 0.001})
    cost_ = sess.run(cost, feed_dict={ X:X_train_mini, Y: y_train_mini, learning_rate: 0.001})
    cost_history = np.append(cost_history, cost_)
    if (epoch % 50 == 0):
        print("Reached epoch",epoch,"cost J =", cost_)


Reached epoch 0 cost J = 0.3247578
Reached epoch 50 cost J = 0.2785224
Reached epoch 100 cost J = 0.23683165


In [26]:
# Accuracy
correct_predictions = tf.equal(tf.argmax(y_,0), tf.argmax(Y,0))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))
print ("Accuracy:", accuracy.eval({X: train, Y: labels_, learning_rate:0.001}, session = sess))

Accuracy: 0.35521665


In [11]:
# model
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

def model(minibatch_size, training_epochs, features, classes, logging_step = 100, learning_r = 0.001):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    cost_history = []
    for epoch in range(training_epochs+1):
        for i in range(0, 60000, minibatch_size):
            X_train_mini = features[:, i:i+minibatch_size]
            y_train_mini = classes[:, i:i+minibatch_size]
            sess.run(optimizer, feed_dict = {X: X_train_mini,Y: y_train_mini,learning_rate: learning_r})
            cost_ = sess.run(cost, feed_dict={ X:X_train_mini, Y: y_train_mini, learning_rate: learning_r})
            cost_history = np.append(cost_history, cost_)
        if (epoch % logging_step == 0):
            print("Reached epoch",epoch,"cost J =", cost_)
    correct_predictions = tf.equal(tf.argmax(y_,0), tf.argmax(Y,0))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))
    print ("Accuracy:", accuracy.eval({X: features, Y: classes, learning_rate:learning_r}, session = sess))
    return sess, cost_history


In [12]:
# thêm layer
def create_layer (X, n, activation):
    ndim = int(X.shape[0]) # số chiều của ma trận X
    stddev = 2 / np.sqrt(ndim) # độ lệch chuẩn
    initialization = tf.truncated_normal((n, ndim), stddev = stddev) # khởi tạo ma trận trọng số
    W = tf.Variable(initialization) # khởi tạo trọng số
    b = tf.Variable(tf.zeros([n,1])) # khởi tạo bias
    Z = tf.matmul(W,X)+b # tính Z
    return activation(Z)

In [13]:
n_dim = 784
n1 = 300
n2 = 300
n_outputs = 10
X = tf.placeholder(tf.float32, [n_dim, None])
Y = tf.placeholder(tf.float32, [10, None])
learning_rate = tf.placeholder(tf.float32, shape=()) # learning rate
hidden1 = create_layer (X, n1, activation = tf.nn.relu)
hidden2 = create_layer (hidden1, n2, activation = tf.nn.relu)
outputs = create_layer (hidden2, n_outputs, activation = tf.identity) # layer 3
y_ = tf.nn.softmax(outputs) # softmax
cost = - tf.reduce_mean(Y * tf.log(y_)+(1-Y) * tf.log(1-y_)) # cost function
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) # optimizer

In [69]:
# run model
sess, cost_history = model(100, 100, train, labels_, logging_step = 50, learning_r = 0.001)

Reached epoch 0 cost J = 0.43582654
Reached epoch 50 cost J = 0.32179788
Reached epoch 100 cost J = 0.3119525
Accuracy: 0.8088167


In [73]:
# 50 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 50, train, labels_, logging_step = 50, learning_r = 0.001)

Reached epoch 0 cost J = 0.4703095
Reached epoch 50 cost J = 0.32860017
Accuracy: 0.78328335


In [70]:
# 150 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 150, train, labels_, logging_step = 50, learning_r = 0.001)

Reached epoch 0 cost J = 0.4534734
Reached epoch 50 cost J = 0.32354823
Reached epoch 100 cost J = 0.31442282
Reached epoch 150 cost J = 0.30993968
Accuracy: 0.8242667


In [71]:
# 200 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 200, train, labels_, logging_step = 50, learning_r = 0.001)

Reached epoch 0 cost J = 0.43224895
Reached epoch 50 cost J = 0.32067603
Reached epoch 100 cost J = 0.31121284
Reached epoch 150 cost J = 0.3060673
Reached epoch 200 cost J = 0.30262154
Accuracy: 0.8311


In [74]:
# 250 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 250, train, labels_, logging_step = 50, learning_r = 0.001)

Reached epoch 0 cost J = 0.4417857
Reached epoch 50 cost J = 0.3222163
Reached epoch 100 cost J = 0.31078932
Reached epoch 150 cost J = 0.3050323
Reached epoch 200 cost J = 0.30128682
Reached epoch 250 cost J = 0.29870793
Accuracy: 0.83785


In [75]:
# 300 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 300, train, labels_, logging_step = 50, learning_r = 0.001)

Reached epoch 0 cost J = 0.4407758
Reached epoch 50 cost J = 0.3240612
Reached epoch 100 cost J = 0.3119966
Reached epoch 150 cost J = 0.30650178
Reached epoch 200 cost J = 0.3029989
Reached epoch 250 cost J = 0.3005179
Reached epoch 300 cost J = 0.2983929
Accuracy: 0.8423667


In [76]:
# 400 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 400, train, labels_, logging_step = 50, learning_r = 0.001)

Reached epoch 0 cost J = 0.44144398
Reached epoch 50 cost J = 0.32281595
Reached epoch 100 cost J = 0.3119016
Reached epoch 150 cost J = 0.30625203
Reached epoch 200 cost J = 0.30265903


In [ ]:
# 500 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 500, train, labels_, logging_step = 50, learning_r = 0.001)

In [ ]:
# 600 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 600, train, labels_, logging_step = 50, learning_r = 0.001)

In [ ]:
# 700 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 700, train, labels_, logging_step = 50, learning_r = 0.001)

In [ ]:
# 800 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 800, train, labels_, logging_step = 50, learning_r = 0.001)

In [ ]:
# 900 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 900, train, labels_, logging_step = 50, learning_r = 0.001)

In [15]:
# 1000 epochs , minibatch size = 100, learning rate = 0.001
sess, cost_history = model(100, 1000, train, labels_, logging_step = 50, learning_r = 0.001)

Reached epoch 0 cost J = 0.4367229
Reached epoch 50 cost J = 0.32442304
Reached epoch 100 cost J = 0.31354278
Reached epoch 150 cost J = 0.3077331
Reached epoch 200 cost J = 0.30409515
Reached epoch 250 cost J = 0.30133617
Reached epoch 300 cost J = 0.29917875
Reached epoch 350 cost J = 0.29732043
Reached epoch 400 cost J = 0.29568794
Reached epoch 450 cost J = 0.2942772
Reached epoch 500 cost J = 0.2930462
Reached epoch 550 cost J = 0.29197603
Reached epoch 600 cost J = 0.29106826
Reached epoch 650 cost J = 0.29022777
Reached epoch 700 cost J = 0.2894107
Reached epoch 750 cost J = 0.28867462
Reached epoch 800 cost J = 0.28798804
Reached epoch 850 cost J = 0.28740388
Reached epoch 900 cost J = 0.28687373
Reached epoch 950 cost J = 0.2863618
Reached epoch 1000 cost J = 0.2859032
Accuracy: 0.8711333


In [14]:
# 2000 epochs , minibatch size = 100, learning rate = 0.001
# logging_step là cứ mỗi 50 lần thì sẽ đưa ra một cost
sess2000, cost_history2000 = model(100, 2000, train, labels_, logging_step = 50, learning_r = 0.001)

Reached epoch 0 cost J = 0.43180358
Reached epoch 50 cost J = 0.3209914
Reached epoch 100 cost J = 0.31247938
Reached epoch 150 cost J = 0.30781788
Reached epoch 200 cost J = 0.3046175
Reached epoch 250 cost J = 0.30204326
Reached epoch 300 cost J = 0.2999763
Reached epoch 350 cost J = 0.29834226
Reached epoch 400 cost J = 0.29687488
Reached epoch 450 cost J = 0.29551688
Reached epoch 500 cost J = 0.29434285
Reached epoch 550 cost J = 0.2932459
Reached epoch 600 cost J = 0.29226613
Reached epoch 650 cost J = 0.2912914
Reached epoch 700 cost J = 0.29039
Reached epoch 750 cost J = 0.28957483
Reached epoch 800 cost J = 0.2888287
Reached epoch 850 cost J = 0.28818712
Reached epoch 900 cost J = 0.28757888
Reached epoch 950 cost J = 0.28703883
Reached epoch 1000 cost J = 0.28655422
Reached epoch 1050 cost J = 0.28610343
Reached epoch 1100 cost J = 0.28566012
Reached epoch 1150 cost J = 0.28524348
Reached epoch 1200 cost J = 0.2848638
Reached epoch 1250 cost J = 0.2845134
Reached epoch 1300 c